In [1]:
!wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip

--2024-10-15 20:22:22--  https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: 'bank+marketing.zip.1'

     0K .......... .......... .......... .......... .......... 24.6K
    50K .......... .......... .......... .......... .......... 21.5K
   100K .......... .......... .......... .......... .......... 17.2K
   150K .......... .......... .......... .......... .......... 24.6K
   200K .......... .......... .......... .......... .......... 34.5K
   250K .......... .......... .......... .......... .......... 21.5K
   300K .......... .......... .......... .......... .......... 11.2K
   350K .......... .......... .......... .......... .......... 14.5K
   400K .......... .......... .......... .......... .......... 21.1K
   450K .......... ........

In [1]:
import numpy as np
import pandas as pd
import shutil

from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score, accuracy_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [16]:
shutil.unpack_archive('bank+marketing.zip')


In [17]:
shutil.unpack_archive('bank.zip')

In [2]:
df = pd.read_csv('bank-full.csv', sep =';')

In [3]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [4]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [5]:
df = df[['age','job','marital','education','balance','housing', 'contact','day','month','duration','campaign','pdays','previous','poutcome','y']]

In [6]:
df

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,825,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,1729,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,5715,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,668,no,telephone,17,nov,508,4,-1,0,unknown,no


Check if missing features

In [7]:
df.isnull().sum()

age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

No missing values!

## 1. What is the most frequent observation (mode) for the column education?

In [8]:
df['education'].mode()

0    secondary
Name: education, dtype: object

## 2. Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

- age and balance
- day and campaign
- day and pdays
- pdays and previous

In [9]:
df.dtypes

age           int64
job          object
marital      object
education    object
balance       int64
housing      object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [10]:
# df.age

In [11]:
df['age'].corr(df['balance'])

0.0977827393713475

In [12]:
df['day'].corr(df['campaign'])

0.16249021632619273

In [13]:
df['day'].corr(df['pdays'])

-0.09304407377294055

In [14]:
df['pdays'].corr(df['previous'])

0.45481963548050097

The correlation is highest between pdays and previous. However, better to create a matrix of the numerical features only.

In [15]:
df_numeric = df[['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']]

In [16]:
df_numeric.corr()

,age,balance,day,duration,campaign,pdays,previous
age,1.000000,0.097783,-0.009120,-0.004648,0.004760,-0.023758,0.001288
balance,0.097783,1.000000,0.004503,0.021560,-0.014578,0.003435,0.016674
day,-0.009120,0.004503,1.000000,-0.030206,0.162490,-0.093044,-0.051710
duration,-0.004648,0.021560,-0.030206,1.000000,-0.084570,-0.001565,0.001203
campaign,0.004760,-0.014578,0.162490,-0.084570,1.000000,-0.088628,-0.032855
pdays,-0.023758,0.003435,-0.093044,-0.001565,-0.088628,1.000000,0.454820
previous,0.001288,0.016674,-0.051710,0.001203,-0.032855,0.454820,1.000000


In [17]:
df

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,825,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,1729,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,5715,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,668,no,telephone,17,nov,508,4,-1,0,unknown,no


In [18]:
df.y = (df.y=='yes').astype(int)

C:\Users\ellen\AppData\Local\Temp\ipykernel_25692\1365307129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.y = (df.y=='yes').astype(int)


In [19]:
df

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,825,no,cellular,17,nov,977,3,-1,0,unknown,1
45207,71,retired,divorced,primary,1729,no,cellular,17,nov,456,2,-1,0,unknown,1
45208,72,retired,married,secondary,5715,no,cellular,17,nov,1127,5,184,3,success,1
45209,57,blue-collar,married,secondary,668,no,telephone,17,nov,508,4,-1,0,unknown,0


In [20]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

In [21]:
len(df), len(df_train), len(df_val), len(df_test)

(45211, 27126, 9042, 9043)

In [22]:
len(df_train) + len(df_val) + len(df_test)

45211

In [23]:
len(df) * 0.2

9042.2

In [24]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [25]:
y_train = df_train.y
y_val = df_val.y

del df_train['y']
del df_val['y']

## 3. Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
Round the scores to 2 decimals using round(score, 2).

In [26]:
df.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,0


In [27]:
numerical = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
categorical = ['job', 'marital', 'education', 'housing', 'contact','month', 'poutcome']
df.columns

Index(['age', 'job', 'marital', 'education', 'balance', 'housing', 'contact',
       'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome',
       'y'],
      dtype='object')

In [28]:
def multi_info_score(series):
    return mutual_info_score(series,y_train)


In [29]:
# In need to do round to three to see
mi = df_train[categorical].apply(multi_info_score)
mi_round = mi.round(3)
mi_round.sort_values(ascending = False)

poutcome     0.030
month        0.025
contact      0.013
housing      0.010
job          0.007
education    0.003
marital      0.002
dtype: float64

The highest mutual information score is obtained by "poutcome"

## Question 4
## Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. 
- Include them using one-hot encoding.
- Fit the model on the training dataset.
- To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [30]:
categorical

['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

In [31]:
categorical

['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

In [32]:
numerical

['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [33]:
dicts = df_train[numerical+categorical].to_dict(orient='records')
# dicts

In [34]:
dv = DictVectorizer(sparse = False)

In [35]:
dv.fit(dicts)

DictVectorizer(sparse=False)

In [36]:
dv.get_feature_names_out()

array(['age', 'balance', 'campaign', 'contact=cellular',
       'contact=telephone', 'contact=unknown', 'day', 'duration',
       'education=primary', 'education=secondary', 'education=tertiary',
       'education=unknown', 'housing=no', 'housing=yes', 'job=admin.',
       'job=blue-collar', 'job=entrepreneur', 'job=housemaid',
       'job=management', 'job=retired', 'job=self-employed',
       'job=services', 'job=student', 'job=technician', 'job=unemployed',
       'job=unknown', 'marital=divorced', 'marital=married',
       'marital=single', 'month=apr', 'month=aug', 'month=dec',
       'month=feb', 'month=jan', 'month=jul', 'month=jun', 'month=mar',
       'month=may', 'month=nov', 'month=oct', 'month=sep', 'pdays',
       'poutcome=failure', 'poutcome=other', 'poutcome=success',
       'poutcome=unknown', 'previous'], dtype=object)

In [37]:
X_train = dv.transform(dicts)

In [38]:
val_dicts = df_val[numerical+categorical].to_dict(orient='records')

In [39]:
X_validation = dv.transform(val_dicts)

In [40]:
# X_train

In [41]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [42]:
model.coef_.round(3)

array([[-2.000e-03,  0.000e+00, -8.000e-02,  2.810e-01,  7.300e-02,
        -1.297e+00,  7.000e-03,  4.000e-03, -4.340e-01, -2.410e-01,
        -6.100e-02, -2.080e-01, -1.050e-01, -8.380e-01,  6.800e-02,
        -2.290e-01, -2.610e-01, -2.990e-01, -8.500e-02,  3.330e-01,
        -3.000e-01, -1.530e-01,  2.580e-01, -1.510e-01,  1.700e-02,
        -1.410e-01, -3.010e-01, -4.590e-01, -1.830e-01,  4.600e-02,
        -7.100e-01,  3.360e-01, -3.380e-01, -1.042e+00, -9.940e-01,
         3.070e-01,  1.359e+00, -4.970e-01, -9.740e-01,  8.000e-01,
         7.650e-01, -1.000e-03, -7.890e-01, -6.230e-01,  1.496e+00,
        -1.025e+00,  6.000e-03]])

In [43]:
model.intercept_[0]

-0.9426328016047085

In [44]:
y_pred_val = model.predict(X_validation)

In [45]:
y_pred_prob = model.predict_proba(X_validation)[:,1]
decision = (y_pred_prob >= 0.5)

In [46]:
decision

array([False, False, False, ..., False, False, False])

In [47]:
df_predict = pd.DataFrame()
df_predict['probability'] = decision.astype(int) 
df_predict['actual'] = y_val
df_predict['cor'] = df_predict['probability'] == df_predict['actual']

In [48]:
df_predict

,probability,actual,cor
0,0,0,True
1,0,0,True
2,0,1,False
3,0,0,True
4,0,1,False
...,...,...,...
9037,0,0,True
9038,0,1,False
9039,0,0,True
9040,0,0,True


In [49]:
df_predict['cor'].mean()

0.9015704490157045

## Question 5: Let's find the least useful feature using the feature elimination technique.
- Train a model with all these features (using the same parameters as in Q4).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

In [50]:
total_list = numerical + categorical

In [51]:
numerical

['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [52]:
categorical

['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

Which of following feature has the smallest difference?

- age
- balance
- marital
- previous

In [57]:
numerical

['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [62]:
numerical_copy = numerical.copy()
numerical_min_age = numerical_copy.remove('age')

In [64]:
test_drop = pd.DataFrame(columns=['feature', 'accuracy'])

## Question 6: Now let's train a regularized logistic regression.
- Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.